In [1]:
import numpy as np
import sympy as sp
from math import pi
from math import cos
from math import sin

In [6]:
CODIGO = 2200273
#UNIDADES (Metros)
larg_cables = 1.5
#UNIDADES (Pulgadas)
diam_cables = 1 / 4
#UNIDADES (Milimetros cuadrados)
area_cables = np.pi * (diam_cables * 25.4 / 1000) ** 2 / 4
#UNIDADES (Pulgadas)
diam_pasadores= 1 / 2
#UNIDADES (Milimetros cuadrados)
area_pasadores= np.pi * (diam_pasadores * 25.4 / 1000) ** 2 / 4
#El área de los pasadores los divido sobre 1000 para obtenerlos directamente en metros
X = 14
#en metros
L = 0.4

FacSeguridad_fluencia = float(f"1.{X}")
FacSeguridad_ultima_resistencia = float(f"1.{X + 3}")

#para convertir los ángulos a radianes
anguloBC = 40 * np.pi / 180
anguloBE = 50 * np.pi / 180


NameError: name 'np' is not defined

In [ ]:
#Datos del material
#UNIDADES (MPa)
Esfuerzo_ultimo = 400
Esfuerzo_fluencia_traccion = 250
Esfuerzo_fluencia_cortante = 145
#UNIDADES (Gpa)
E = 200

In [10]:
# Ecuaciones de Equilibrio
#Definimos las variables necesarias y definimos una a una las ecuaciones de la estática
Ay, Ax, BC, BD, BE, W = sp.symbols("Ax Ay BC BD BE W")
#En x actúa la reacción de A, y las componentes respectivas de BC y CE 
sumfuerza_en_x = Ax - BC * sp.sin(anguloBC) + BE * sp.sin(anguloBE)
equilibrioX = sp.Eq(sumafuerzas_en_x,0)
equilibrioX

Eq(Ay - 0.642787609686539*BC + 0.766044443118978*BE, 0)

In [9]:
#En y actúa la carga puntual de W (W*L), la reacción de A, el cable D y las componentes de BC y BE
sumafuerza_en_y = Ay - W * L + BC * sp.cos(anguloBC) + BE * sp.cos(anguloBE) + BD
equilibrioY = sp.Eq(sumafuerzas_en_y, 0)
equilibrioY

Eq(Ax + 0.766044443118978*BC + BD + 0.642787609686539*BE - 2.8*W, 0)

In [11]:
#Se hizo momento en A, de manera que hacen momento las componentes de los cables BC y BE, el cable D, y la puntual de W*L
Momento_en_A = - W * L ** 2 / 2 + BC * sp.cos(anguloBC) * L + BE * sp.cos(anguloBE) * L + BD * L
equilibrioMomentos = sp.Eq(Momento_en_A, 0)
equilibrioMomentos

Eq(2.14492444073314*BC + 2.8*BD + 1.79980530712231*BE - 3.92*W, 0)

In [16]:
#Hallar ecuacion de compatibilidad
Equivalencia_BC = sp.Eq(BC - BD * sp.cos(anguloBC), 0)
Ecuacion_equivalencia_BC = BD * sp.cos(anguloBC)
Equivalencia_BC

Eq(BC - 0.766044443118978*BD, 0)

In [33]:
#Ecuación de compatibilidad para BE
Equivalencia_BE = sp.Eq(BE - BD * sp.cos(anguloBE), 0)
Ecuacion_equivalencia_BE = BD * sp.cos(anguloBE)
Equivalencia_BE

Eq(-0.642787609686539*BD + BE, 0)

In [35]:
BD_EQ_MOM = sp.Eq(Momento_en_A.subs([(BC,Ecuacion_equivalencia_BC), (BE,Ecuacion_equivalencia_BE)]), 0)

In [39]:
BD_W = sp.solve(BD_EQ_MOM)[0][BD]
BD_W

0.7*W

In [40]:
#Definimos el cortante doble para los apoyos en términos de W
AyEq = sumafuerzas_en_y.subs([(BC, Ecuacion_equivalencia_BC), (BE, Ecuacion_equivalencia_BE), (BD,BD_W)])
Ay_W = sp.solve(sp.Eq(AyEq, 0))
Ay_W = Ay_W[0][Ay]
Ay_W

1.4*W

In [41]:
AxEq = sumafuerzas_en_x.subs([(BC, Ecuacion_equivalencia_BC), (BE, Ecuacion_equivalencia_BE), (BD, BD_W)])
Ax_W = sp.solve(sp.Eq(AxEq, 0))
Ax_W = Ax_W[0][Ax]
Ax_W

-7.7715611723761e-17*W

In [42]:
#Apoyo en A
Apoyo_A = sp.sqrt(Ay_W ** 2 + Ax_W ** 2)
Apoyo_A

1.4*sqrt(W**2)

In [45]:
+#Punto de union de los cables
FxCortanteDoble = Ecuacion_equivalencia_BC.subs(BD, BD_W) * sp.sin(anguloBE) - Ecuacion_equivalencia_BC.subs(BD, BD_W)
FxCortanteDoble = Ecuacion_equivalencia_BC.subs(BD, BD_W) * sp.sin(anguloBE) - Ecuacion_equivalencia_BC.subs(BD, BD_W)
FyCortanteDoble = Ecuacion_equivalencia_BC.subs(BD, BD_W) * sp.cos(anguloBE) - Ecuacion_equivalencia_BC.subs(BD, BD_W)
Apoyo_B= sp.sqrt(FxCortanteDoble ** 2 + FyCortanteDoble ** 2)
Apoyo_B

0.228975013030582*sqrt(W**2)

In [44]:
FBC = Ecuacion_equivalencia_BC.subs(BD, BD_W)
FBE = Ecuacion_equivalencia_BE.subs(BD, BD_W)
Vector_de_Fuerzas = [FBC, BD_W, FBE]
Vector_de_Fuerzas

[0.536231110183285*W, 0.7*W, 0.449951326780578*W]

In [54]:
Wmax = []
for fuerza in Vector_de_Fuerzas:
    #Esfuerzo normal en cables
    Esf_normal = sp.Eq(fuerza / area_cables, 400 * 10 ** 6 / FacSeguridad_ultima_resistencia)
    W_cables = sp.solve(Esf_normal)[0] / 1000
    #Esfuerzo cortante simple en pasadores
    Esf_cortante = sp.Eq(fuerza / area_pasadores, 145 * 10 ** 6 / FacSeguridad_fluencia)
    W_pasadores = sp.solve(esf_cortante)[0] / 1000
    Wmax.append(W_cables)
    Wmax.append(W_pasadores)
    
print("Carga dsitribuida máxima en los cables [kN / m]")
min(Wmax)

Carga dsitribuida máxima en los cables [kN / m]


15.4672612667124

In [55]:
#En teoría, ya se calculó el valor de w = 15,46 kN/m aproximadamente. Ahora podremos calcular las fuerzas y las...
#...deformaciones para el inciso B

In [75]:
Fuerza_BE = Ecuacion_equivalencia_BE.subs([(BD, BD_W), (W, Wmax)])
Fuerza_BE * 15.4672612667124

6.95951472861908*W

In [74]:
Deformacion_BE = (Fuerza_BE * 1000 * larg_cables) / (area_cables * 200 * 10 ** 9)
Deformacion_BE * 15.4672612667124

0.00164817335817062*W

In [79]:
Fuerza_BC = Ecuacion_equivalencia_BC.subs([(BD, BD_W), (W, Wmax)])
Fuerza_BC * 15.4672612667124

8.29402668054411*W

In [80]:
Deformacion_BC = (Fuerza_BC * 1000 * long_cables) / (area_cables * 200 * 10 ** 9)
Deformacion_BC

0.000126991875739834*W

In [82]:
Fuerza_BD = BD_W.subs(W, Wmax)
Fuerza_BD * 15.4672612667124

0.7*W

In [84]:
Deformacion_BD = (Fuerza_BD * 1000 * larg_cables) / (area_cables * 200 * 10 ** 9)
Deformacion_BD

0.000165776120276759*W

In [85]:
#Se obtuvieron los valores para las fuerzas y deformaciones en cada cable. A pesar de que quedaron en términos de W, sin embargo
#ya se llegó a que W = 15.4672612667124. Ahora se calcularán las reacciones Ax y Ay partiendo de las ecuaciones descritas al
#inicio

In [92]:
Reaccion_Ax = sumafuerzas_en_x.subs([(BE, Fuerza_BE), (BC, Fuerza_BC)])
Reaccion_Ax = sp.solve(sp.Eq(Reaccion_Ax, 0))[0]
Reaccion_Ax

#El resultado que muestra, es el valor de Ax

{Ay: -5.55111512312578e-17*W}

In [91]:
Reaccion_Ay = sumafuerzas_en_y.subs([(BE, Fuerza_BE), (BC, Fuerza_BC), (BD, Fuerza_BD), (W, Wmax)])
Reaccion_Ay = sp.solve(sp.Eq(Reaccion_Ay, 0))[0]
Reaccion_Ay

#El resultado que muestra, es el valor de Ay

{Ax: 1.4*W}